In [1]:
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp
from dataclasses import dataclass
import os.path as path
from haikunator import Haikunator


In [2]:
fashion_mnist = tf.keras.datasets.fashion_mnist

(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [3]:
@dataclass
class Hyperparams:
    num_units = 0
    dropout = 0.
    optimizer = None

In [4]:
HP_NUM_UNITS = hp.HParam("num_units", hp.Discrete([16, 32]))
HP_DROPOUT = hp.HParam("dropout", hp.RealInterval(0.1, 0.2))
HP_OPTIMIZER = hp.HParam("optimizer", hp.Discrete(["adam", "sgd"]))

METRIC_ACCUARACY = "accuracy"

tblogs = path.expanduser("~/mldata/tblogs/hplogging")
with tf.summary.create_file_writer(tblogs).as_default():
    hp.hparams_config(
        hparams=[HP_NUM_UNITS, HP_DROPOUT, HP_OPTIMIZER],
        metrics=[hp.Metric(METRIC_ACCUARACY, display_name="Accuracy")]
    )

In [5]:
def train_test_model(hparams, runid):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(hparams[HP_NUM_UNITS], activation=tf.nn.relu),
        tf.keras.layers.Dropout(hparams[HP_DROPOUT]),
        tf.keras.layers.Dense(10, activation=tf.nn.softmax)
    ])
    model.compile(
        optimizer=hparams[HP_OPTIMIZER],
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )
    tblogs = path.expanduser(f"~/mldata/tblogs/hplogging/{runid}")
    model.fit(x_train, y_train, epochs=2, callbacks=[tf.keras.callbacks.TensorBoard(tblogs), hp.KerasCallback(tblogs, hparams)])
    _, accuracy = model.evaluate(x_test, y_test)
    return accuracy

In [6]:
for num_units in HP_NUM_UNITS.domain.values:
    for dropout_rate in (HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value):
        for optim in HP_OPTIMIZER.domain.values:
            hparams = {
                HP_NUM_UNITS: num_units,
                HP_DROPOUT: dropout_rate,
                HP_OPTIMIZER: optim
            }
            runid = Haikunator().haikunate()
            print(f"--- Starting trial: {runid}")
            print({h.name: hparams[h] for h in hparams})
            accuracy = train_test_model(hparams, runid)
            print(f"Accuracy={accuracy:.3f}")

--- Starting trial: broad-lake-7341
{'num_units': 16, 'dropout': 0.1, 'optimizer': 'adam'}
Train on 60000 samples
Epoch 1/2
60000/60000 [==============================] - 4s 62us/sample - loss: 0.7092 - accuracy: 0.7533
Epoch 2/2
10000/10000 [==============================] - 0s 36us/sample - loss: 0.4599 - accuracy: 0.8426
Accuracy=0.843
--- Starting trial: icy-bird-9467
{'num_units': 16, 'dropout': 0.1, 'optimizer': 'sgd'}
Train on 60000 samples
Epoch 1/2
60000/60000 [==============================] - 3s 53us/sample - loss: 1.0023 - accuracy: 0.6570
Epoch 2/2
10000/10000 [==============================] - 0s 28us/sample - loss: 0.5677 - accuracy: 0.8043
Accuracy=0.804
--- Starting trial: damp-block-4170
{'num_units': 16, 'dropout': 0.2, 'optimizer': 'adam'}
Train on 60000 samples
Epoch 1/2
60000/60000 [==============================] - 3s 48us/sample - loss: 0.7693 - accuracy: 0.7263
Epoch 2/2
10000/10000 [==============================] - 0s 32us/sample - loss: 0.4829 - accuracy: 0.